## package installation and load packages


In [ ]:
### packages required 
!pip install fair-esm 
!pip install torch
!pip install tensorflow
!pip install sklearn
!pip install h5py

In [ ]:
import esm
import pandas as pd
import numpy as np
from keras.layers import Input, Dense, Activation, BatchNormalization, Flatten, Conv1D
from keras.layers import Dropout, AveragePooling1D, MaxPooling1D
from keras.models import Sequential,Model, load_model
from keras.optimizers import SGD
from keras.callbacks import ModelCheckpoint,LearningRateScheduler, EarlyStopping
import keras
from keras import backend as K
import tensorflow as tf
if tf.test.gpu_device_name():
    print('GPU found')
    tf.config.experimental.set_visible_devices(tf.config.list_physical_devices('GPU')[0], 'GPU') # set the deep learning with GPU 
else:
    print("No GPU found")

### peptide embeddings with differen pretrained model
https://github.com/facebookresearch/esm

Explaination of the memeory usage of the following models

sequence length > 900 

2560 output dimension model might need 24 G GPU memory

5129 output dimension model, (in our attempts, 40 GB GPU memory is not enough) 


In [ ]:
def esm_embeddings_320(esm2, esm2_alphabet, peptide_sequence_list):
  # NOTICE: ESM for embeddings is quite RAM usage, if your sequence is too long, 
  #         or you have too many sequences for transformation in a single converting, 
  #         you computer might automatically kill the job.
  import torch
  import esm
  import collections
  import pandas as pd
  import gc

  if torch.cuda.is_available():
    device = torch.device("cuda")
  else:
    device = torch.device("cpu")
  esm2 = esm2.eval().to(device)

  batch_converter = esm2_alphabet.get_batch_converter()

  # load the peptide sequence list into the bach_converter
  batch_labels, batch_strs, batch_tokens = batch_converter(peptide_sequence_list)
  batch_lens = (batch_tokens != alphabet.padding_idx).sum(1)
  ## batch tokens are the embedding results of the whole data set

  batch_tokens = batch_tokens.to(device)

  # Extract per-residue representations (on CPU)
  with torch.no_grad():
      # Here we export the last layer of the EMS model output as the representation of the peptides
      # model'esm2_t6_8M_UR50D' only has 6 layers, and therefore repr_layers parameters is equal to 6
      results = esm2(batch_tokens, repr_layers=[6], return_contacts=False)
  token_representations = results["representations"][6].cpu()

  # Generate per-sequence representations via averaging
  # NOTE: token 0 is always a beginning-of-sequence token, so the first residue is token 1.
  sequence_representations = []
  for i, tokens_len in enumerate(batch_lens):
      sequence_representations.append(token_representations[i, 1 : tokens_len - 1].mean(0))
  # save dataset
  # sequence_representations is a list and each element is a tensor
  embeddings_results = collections.defaultdict(list)
  for i in range(len(sequence_representations)):
      # tensor can be transformed as numpy sequence_representations[0].numpy() or sequence_representations[0].to_list
      each_seq_rep = sequence_representations[i].tolist()
      for each_element in each_seq_rep:
          embeddings_results[i].append(each_element)
  embeddings_results = pd.DataFrame(embeddings_results).T
  del  batch_labels, batch_strs, batch_tokens, results, token_representations
  torch.cuda.empty_cache()
  gc.collect()
  return embeddings_results


In [ ]:
def esm_embeddings_480(esm2, esm2_alphabet, peptide_sequence_list):
  # NOTICE: ESM for embeddings is quite RAM usage, if your sequence is too long, 
  #         or you have too many sequences for transformation in a single converting, 
  #         you computer might automatically kill the job.
  import torch
  import esm
  import collections
  import pandas as pd
  import gc

  if torch.cuda.is_available():
    device = torch.device("cuda")
  else:
    device = torch.device("cpu")
  esm2 = esm2.eval().to(device)

  batch_converter = esm2_alphabet.get_batch_converter()

  # load the peptide sequence list into the bach_converter
  batch_labels, batch_strs, batch_tokens = batch_converter(peptide_sequence_list)
  batch_lens = (batch_tokens != alphabet.padding_idx).sum(1)
  ## batch tokens are the embedding results of the whole data set

  batch_tokens = batch_tokens.to(device)

  # Extract per-residue representations (on CPU)
  with torch.no_grad():
      # Here we export the last layer of the EMS model output as the representation of the peptides
      # model'esm2_t12_35M_UR50D' only has 12 layers, and therefore repr_layers parameters is equal to 12
      results = esm2(batch_tokens, repr_layers=[12], return_contacts=False)
  token_representations = results["representations"][12].cpu()

  # Generate per-sequence representations via averaging
  # NOTE: token 0 is always a beginning-of-sequence token, so the first residue is token 1.
  sequence_representations = []
  for i, tokens_len in enumerate(batch_lens):
      sequence_representations.append(token_representations[i, 1 : tokens_len - 1].mean(0))
  # save dataset
  # sequence_representations is a list and each element is a tensor
  embeddings_results = collections.defaultdict(list)
  for i in range(len(sequence_representations)):
      # tensor can be transformed as numpy sequence_representations[0].numpy() or sequence_representations[0].to_list
      each_seq_rep = sequence_representations[i].tolist()
      for each_element in each_seq_rep:
          embeddings_results[i].append(each_element)
  embeddings_results = pd.DataFrame(embeddings_results).T
  del  batch_labels, batch_strs, batch_tokens, results, token_representations
  torch.cuda.empty_cache()
  gc.collect()
  return embeddings_results


In [ ]:
def esm_embeddings_640(esm2, esm2_alphabet, peptide_sequence_list):
  # NOTICE: ESM for embeddings is quite RAM usage, if your sequence is too long, 
  #         or you have too many sequences for transformation in a single converting, 
  #         you computer might automatically kill the job.
  import torch
  import esm
  import collections
  import pandas as pd
  import gc

  if torch.cuda.is_available():
    device = torch.device("cuda")
  else:
    device = torch.device("cpu")
  esm2 = esm2.eval().to(device)

  batch_converter = esm2_alphabet.get_batch_converter()

  # load the peptide sequence list into the bach_converter
  batch_labels, batch_strs, batch_tokens = batch_converter(peptide_sequence_list)
  batch_lens = (batch_tokens != alphabet.padding_idx).sum(1)
  ## batch tokens are the embedding results of the whole data set

  batch_tokens = batch_tokens.to(device)

  # Extract per-residue representations (on CPU)
  with torch.no_grad():
      # Here we export the last layer of the EMS model output as the representation of the peptides
      # model'esm2_t30_150M_UR50D' only has 30 layers, and therefore repr_layers parameters is equal to 30
      results = esm2(batch_tokens, repr_layers=[30], return_contacts=False)
  token_representations = results["representations"][30].cpu()

  # Generate per-sequence representations via averaging
  # NOTE: token 0 is always a beginning-of-sequence token, so the first residue is token 1.
  sequence_representations = []
  for i, tokens_len in enumerate(batch_lens):
      sequence_representations.append(token_representations[i, 1 : tokens_len - 1].mean(0))
  # save dataset
  # sequence_representations is a list and each element is a tensor
  embeddings_results = collections.defaultdict(list)
  for i in range(len(sequence_representations)):
      # tensor can be transformed as numpy sequence_representations[0].numpy() or sequence_representations[0].to_list
      each_seq_rep = sequence_representations[i].tolist()
      for each_element in each_seq_rep:
          embeddings_results[i].append(each_element)
  embeddings_results = pd.DataFrame(embeddings_results).T
  del  batch_labels, batch_strs, batch_tokens, results, token_representations
  torch.cuda.empty_cache()
  gc.collect()
  return embeddings_results


In [ ]:
def esm_embeddings_1280(esm2, esm2_alphabet, peptide_sequence_list):
  # NOTICE: ESM for embeddings is quite RAM usage, if your sequence is too long, 
  #         or you have too many sequences for transformation in a single converting, 
  #         you computer might automatically kill the job.
  import torch
  import esm
  import collections
  import pandas as pd
  import gc

  if torch.cuda.is_available():
    device = torch.device("cuda")
  else:
    device = torch.device("cpu")
  esm2 = esm2.eval().to(device)

  batch_converter = esm2_alphabet.get_batch_converter()

  # load the peptide sequence list into the bach_converter
  batch_labels, batch_strs, batch_tokens = batch_converter(peptide_sequence_list)
  batch_lens = (batch_tokens != alphabet.padding_idx).sum(1)
  ## batch tokens are the embedding results of the whole data set

  batch_tokens = batch_tokens.to(device)

  # Extract per-residue representations (on CPU)
  with torch.no_grad():
      # Here we export the last layer of the EMS model output as the representation of the peptides
      # model'esm2_t33_650M_UR50D' only has 33 layers, and therefore repr_layers parameters is equal to 33
      results = esm2(batch_tokens, repr_layers=[33], return_contacts=False)
  token_representations = results["representations"][33].cpu()

  # Generate per-sequence representations via averaging
  # NOTE: token 0 is always a beginning-of-sequence token, so the first residue is token 1.
  sequence_representations = []
  for i, tokens_len in enumerate(batch_lens):
      sequence_representations.append(token_representations[i, 1 : tokens_len - 1].mean(0))
  # save dataset
  # sequence_representations is a list and each element is a tensor
  embeddings_results = collections.defaultdict(list)
  for i in range(len(sequence_representations)):
      # tensor can be transformed as numpy sequence_representations[0].numpy() or sequence_representations[0].to_list
      each_seq_rep = sequence_representations[i].tolist()
      for each_element in each_seq_rep:
          embeddings_results[i].append(each_element)
  embeddings_results = pd.DataFrame(embeddings_results).T
  del  batch_labels, batch_strs, batch_tokens, results, token_representations
  torch.cuda.empty_cache()
  gc.collect()
  return embeddings_results


In [ ]:
def esm_embeddings_2560(esm2, esm2_alphabet, peptide_sequence_list):
  # NOTICE: ESM for embeddings is quite RAM usage, if your sequence is too long, 
  #         or you have too many sequences for transformation in a single converting, 
  #         you computer might automatically kill the job.
  import torch
  import esm
  import collections
  import pandas as pd
  import gc

  if torch.cuda.is_available():
    device = torch.device("cuda")
  else:
    device = torch.device("cpu")
  esm2 = esm2.eval().to(device)

  batch_converter = esm2_alphabet.get_batch_converter()

  # load the peptide sequence list into the bach_converter
  batch_labels, batch_strs, batch_tokens = batch_converter(peptide_sequence_list)
  batch_lens = (batch_tokens != alphabet.padding_idx).sum(1)
  ## batch tokens are the embedding results of the whole data set

  batch_tokens = batch_tokens.to(device)

  # Extract per-residue representations (on CPU)
  with torch.no_grad():
      # Here we export the last layer of the EMS model output as the representation of the peptides
      # model'esm2_t36_3B_UR50D' only has 36 layers, and therefore repr_layers parameters is equal to 36
      results = esm2(batch_tokens, repr_layers=[36], return_contacts=False)
  token_representations = results["representations"][36].cpu()

  # Generate per-sequence representations via averaging
  # NOTE: token 0 is always a beginning-of-sequence token, so the first residue is token 1.
  sequence_representations = []
  for i, tokens_len in enumerate(batch_lens):
      sequence_representations.append(token_representations[i, 1 : tokens_len - 1].mean(0))
  # save dataset
  # sequence_representations is a list and each element is a tensor
  embeddings_results = collections.defaultdict(list)
  for i in range(len(sequence_representations)):
      # tensor can be transformed as numpy sequence_representations[0].numpy() or sequence_representations[0].to_list
      each_seq_rep = sequence_representations[i].tolist()
      for each_element in each_seq_rep:
          embeddings_results[i].append(each_element)
  embeddings_results = pd.DataFrame(embeddings_results).T
  del  batch_labels, batch_strs, batch_tokens, results, token_representations
  torch.cuda.empty_cache()
  gc.collect()
  return embeddings_results


In [ ]:
def esm_embeddings_5120(esm2, esm2_alphabet, peptide_sequence_list):
  # NOTICE: ESM for embeddings is quite RAM usage, if your sequence is too long, 
  #         or you have too many sequences for transformation in a single converting, 
  #         you computer might automatically kill the job.
  import torch
  import esm
  import collections
  import pandas as pd
  import gc

  if torch.cuda.is_available():
    device = torch.device("cuda")
  else:
    device = torch.device("cpu")
  esm2 = esm2.eval().to(device)

  batch_converter = esm2_alphabet.get_batch_converter()

  # load the peptide sequence list into the bach_converter
  batch_labels, batch_strs, batch_tokens = batch_converter(peptide_sequence_list)
  batch_lens = (batch_tokens != alphabet.padding_idx).sum(1)
  ## batch tokens are the embedding results of the whole data set

  batch_tokens = batch_tokens.to(device)

  # Extract per-residue representations (on CPU)
  with torch.no_grad():
      # Here we export the last layer of the EMS model output as the representation of the peptides
      # model'esm2_t48_15B_UR50D' only has 48 layers, and therefore repr_layers parameters is equal to 48
      results = esm2(batch_tokens, repr_layers=[48], return_contacts=False)
  token_representations = results["representations"][48].cpu()

  # Generate per-sequence representations via averaging
  # NOTE: token 0 is always a beginning-of-sequence token, so the first residue is token 1.
  sequence_representations = []
  for i, tokens_len in enumerate(batch_lens):
      sequence_representations.append(token_representations[i, 1 : tokens_len - 1].mean(0))
  # save dataset
  # sequence_representations is a list and each element is a tensor
  embeddings_results = collections.defaultdict(list)
  for i in range(len(sequence_representations)):
      # tensor can be transformed as numpy sequence_representations[0].numpy() or sequence_representations[0].to_list
      each_seq_rep = sequence_representations[i].tolist()
      for each_element in each_seq_rep:
          embeddings_results[i].append(each_element)
  embeddings_results = pd.DataFrame(embeddings_results).T
  del  batch_labels, batch_strs, batch_tokens, results, token_representations
  torch.cuda.empty_cache()
  gc.collect()
  return embeddings_results


### connect with googledrive


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!ls

Mounted at /content/drive
drive  sample_data


In [ ]:
import os
os.chdir('/content/drive/MyDrive/universal_allergenicity_new')

In [ ]:
!pwd
!ls

/content/drive/MyDrive/universal_allergenicity_new
allergens_dataset.xlsx
whole_sample_dataset_esm2_t12_35M_UR50D_unified_480_dimension.csv
whole_sample_dataset_esm2_t30_150M_UR50D_unified_640_dimension.csv
whole_sample_dataset_esm2_t6_8M_UR50D_unified_320_dimension.csv


#### load packages

In [ ]:
from keras.layers import Input, Dense, Activation, BatchNormalization, Flatten, Conv1D
from keras.layers import Dropout, AveragePooling1D, MaxPooling1D
from keras.models import Sequential,Model, load_model
from keras.optimizers import SGD
from keras.callbacks import ModelCheckpoint,LearningRateScheduler, EarlyStopping
import keras
from keras import backend as K
import tensorflow as tf
if tf.test.gpu_device_name():
    print('GPU found')
    tf.config.experimental.set_visible_devices(tf.config.list_physical_devices('GPU')[0], 'GPU') # set the deep learning with GPU 
else:
    print("No GPU found")

## Sequence embeddings with different pretrained protein language models

### 320 feature dimension embedding test

In [ ]:
import numpy as np
import pandas as pd
import esm
# select the ESM model for embeddings (you can select you desired model from https://github.com/facebookresearch/esm)
# NOTICE: if you choose other model, the following model architecture might not be very compitable
#         bseides,please revise the correspdoning parameters in esm_embeddings function (layers for feature extraction)
model, alphabet = esm.pretrained.esm2_t6_8M_UR50D()


# whole dataset loading and dataset splitting 
dataset = pd.read_excel('allergens_dataset.xlsx',na_filter = False) # take care the NA sequence problem

# generate the peptide embeddings
sequence_list = dataset['sequence'] 
embeddings_results = pd.DataFrame()
for seq in sequence_list:
    # the setting is just following the input format setting in ESM model, [name,sequence]
    tuple_sequence = tuple([seq,seq])
    peptide_sequence_list = []
    peptide_sequence_list.append(tuple_sequence) # build a summarize list variable including all the sequence information
    # employ ESM model for converting and save the converted data in csv format
    one_seq_embeddings = esm_embeddings_320(model, alphabet, peptide_sequence_list)
    embeddings_results= pd.concat([embeddings_results,one_seq_embeddings])
embeddings_results.to_csv('whole_sample_dataset_esm2_t6_8M_UR50D_unified_320_dimension.csv')

### 480 feature dimension embedding test

In [ ]:
import numpy as np
import pandas as pd
import esm
# select the ESM model for embeddings (you can select you desired model from https://github.com/facebookresearch/esm)
# NOTICE: if you choose other model, the following model architecture might not be very compitable
#         bseides,please revise the correspdoning parameters in esm_embeddings function (layers for feature extraction)
model, alphabet = esm.pretrained.esm2_t12_35M_UR50D()


# whole dataset loading and dataset splitting 
dataset = pd.read_excel('allergens_dataset.xlsx',na_filter = False) # take care the NA sequence problem

# generate the peptide embeddings
sequence_list = dataset['sequence'] 
embeddings_results = pd.DataFrame()
for seq in sequence_list:
    # the setting is just following the input format setting in ESM model, [name,sequence]
    tuple_sequence = tuple([seq,seq])
    peptide_sequence_list = []
    peptide_sequence_list.append(tuple_sequence) # build a summarize list variable including all the sequence information
    # employ ESM model for converting and save the converted data in csv format
    one_seq_embeddings = esm_embeddings_480(model, alphabet, peptide_sequence_list)
    embeddings_results= pd.concat([embeddings_results,one_seq_embeddings])
embeddings_results.to_csv('whole_sample_dataset_esm2_t12_35M_UR50D_unified_480_dimension.csv')

### 640 feature dimension embedding test

In [ ]:
import numpy as np
import pandas as pd
import esm
# select the ESM model for embeddings (you can select you desired model from https://github.com/facebookresearch/esm)
# NOTICE: if you choose other model, the following model architecture might not be very compitable
#         bseides,please revise the correspdoning parameters in esm_embeddings function (layers for feature extraction)
model, alphabet = esm.pretrained.esm2_t30_150M_UR50D()


# whole dataset loading and dataset splitting 
dataset = pd.read_excel('allergens_dataset.xlsx',na_filter = False) # take care the NA sequence problem

# generate the peptide embeddings
sequence_list = dataset['sequence'] 
embeddings_results = pd.DataFrame()
for seq in sequence_list:
    # the setting is just following the input format setting in ESM model, [name,sequence]
    tuple_sequence = tuple([seq,seq])
    peptide_sequence_list = []
    peptide_sequence_list.append(tuple_sequence) # build a summarize list variable including all the sequence information
    # employ ESM model for converting and save the converted data in csv format
    one_seq_embeddings = esm_embeddings_640(model, alphabet, peptide_sequence_list)
    embeddings_results= pd.concat([embeddings_results,one_seq_embeddings])
embeddings_results.to_csv('whole_sample_dataset_esm2_t30_150M_UR50D_unified_640_dimension.csv')

### 1280 feature dimension embedding test

In [ ]:
import numpy as np
import pandas as pd
import esm
# select the ESM model for embeddings (you can select you desired model from https://github.com/facebookresearch/esm)
# NOTICE: if you choose other model, the following model architecture might not be very compitable
#         bseides,please revise the correspdoning parameters in esm_embeddings function (layers for feature extraction)
model, alphabet = esm.pretrained.esm2_t33_650M_UR50D()

# whole dataset loading and dataset splitting 
dataset = pd.read_excel('allergens_dataset.xlsx',na_filter = False) # take care the NA sequence problem

# generate the peptide embeddings
sequence_list = dataset['sequence'] 
embeddings_results = pd.DataFrame()
for seq in sequence_list:
    # the setting is just following the input format setting in ESM model, [name,sequence]
    tuple_sequence = tuple([seq,seq])
    peptide_sequence_list = []
    peptide_sequence_list.append(tuple_sequence) # build a summarize list variable including all the sequence information
    # employ ESM model for converting and save the converted data in csv format
    one_seq_embeddings = esm_embeddings_1280(model, alphabet, peptide_sequence_list)
    embeddings_results= pd.concat([embeddings_results,one_seq_embeddings])
embeddings_results.to_csv('whole_sample_dataset_esm2_t33_650M_UR50D_unified_1280_dimension.csv')

### 2560 feature dimension embedding test

In [ ]:
import numpy as np
import pandas as pd
import esm
# select the ESM model for embeddings (you can select you desired model from https://github.com/facebookresearch/esm)
# NOTICE: if you choose other model, the following model architecture might not be very compitable
#         bseides,please revise the correspdoning parameters in esm_embeddings function (layers for feature extraction)
model, alphabet = esm.pretrained.esm2_t36_3B_UR50D()


# whole dataset loading and dataset splitting 
dataset = pd.read_excel('allergens_dataset.xlsx',na_filter = False) # take care the NA sequence problem
a=0
# generate the peptide embeddings
sequence_list = dataset['sequence'] 
embeddings_results = pd.DataFrame()
for seq in sequence_list:
    # the setting is just following the input format setting in ESM model, [name,sequence]
    tuple_sequence = tuple([seq,seq])
    peptide_sequence_list = []
    peptide_sequence_list.append(tuple_sequence) # build a summarize list variable including all the sequence information
    # employ ESM model for converting and save the converted data in csv format
    one_seq_embeddings = esm_embeddings_2560(model, alphabet, peptide_sequence_list)
    embeddings_results= pd.concat([embeddings_results,one_seq_embeddings])
    a=a+1
    print(a)
embeddings_results.to_csv('whole_sample_dataset_esm2_t36_3B_UR50D_unified_2560_dimension.csv')

## grid search for hyperparameters of CNN model
Due to the enormous memory consumption and limitted dataset, we do not test 256 fileter size in CNN with 8192 units in dense layer.

### 320 feature dimension embedding test

In [ ]:
import numpy as np
import pandas as pd
import esm

# loading the y dataset for model development 
dataset = pd.read_excel('allergens_dataset.xlsx',na_filter = False) # take care the NA sequence problem
y = dataset['label']
y = np.array(y) # transformed as np.array for CNN model

# read the peptide embeddings
X_data_name = 'whole_sample_dataset_esm2_t6_8M_UR50D_unified_320_dimension.csv'
X_data = pd.read_csv(X_data_name,header=0, index_col = 0,delimiter=',')
X = np.array(X_data)

# split dataset as training and test dataset as ratio of 8:2
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.2, random_state=123)

# normalize the X data range
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train) # normalize X to 0-1 range 
X_test = scaler.transform(X_test)

ACC_collecton = []
CNN_channel = [16,32,64,128,256]
dense_node = [32,64,128,256,512,1024,2048,4096,8192]
kernel_size = [3,6,9,12]
stride_size = [1,2,4,8]
for i in range(len(CNN_channel)):
  for j in range(len(dense_node)):
    for k in range(len(kernel_size)):
      for l in range(len(stride_size)):
        inputShape=(320,1) # input feature size 
        input = Input(inputShape)
        x = Conv1D(CNN_channel[i],(kernel_size[k]),strides = (stride_size[l]),name='layer_conv2',padding='same')(input)
        x = BatchNormalization()(x)
        x = Activation('relu')(x)
        x = MaxPooling1D((2), name='MaxPool2',padding="same")(x)
        x = Dropout(0.15)(x)
        x = Flatten()(x)
        x = Dense(dense_node[j],activation = 'relu',name='fc1')(x)
        x = Dropout(0.15)(x)
        x = Dense(2,activation = 'softmax',name='fc2')(x)
        model = Model(inputs = input,outputs = x,name='Predict')
        # define SGD optimizer
        momentum = 0.5
        sgd = SGD(lr=0.01, momentum=momentum, decay=0.0, nesterov=False)
        # compile the model
        model.compile(loss='sparse_categorical_crossentropy',optimizer=sgd, metrics=['accuracy'])
        # learning deccay setting
        import math
        def step_decay(epoch): # gradually decrease the learning rate 
            initial_lrate=0.1
            drop=0.6
            epochs_drop = 3.0
            lrate= initial_lrate * math.pow(drop,    # math.pow base raised to a power
                  math.floor((1+epoch)/epochs_drop)) # math.floor Round numbers down to the nearest integer
            return lrate
        lr = LearningRateScheduler(step_decay)
        # early stop setting
        early_stop = EarlyStopping(monitor='val_accuracy', patience = 20,verbose=0,restore_best_weights = True)
        # set checkpoint and save the best model
        mc = ModelCheckpoint('best_model_grid_320.h5',  monitor='val_accuracy', mode='max', verbose=0, save_best_only=True, save_weights_only=False)
        # summary the callbacks_list
        callbacks_list = [ lr , early_stop, mc]
        model_history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=200,callbacks=callbacks_list,batch_size = 32, verbose=0)
        # load the save best model
        saved_model = load_model('best_model_grid_320.h5')
        # result collection list
        # confusion matrix 
        predicted_class= []
        predicted_protability = saved_model.predict(X_test,batch_size=1)
        for p in range(predicted_protability.shape[0]):
          index = np.where(predicted_protability[p] == np.amax(predicted_protability[p]))[0][0]
          predicted_class.append(index)
        predicted_class = np.array(predicted_class)
        y_true = y_test    
        from sklearn.metrics import confusion_matrix
        import math
        # np.ravel() return a flatten 1D array
        TP, FP, FN, TN = confusion_matrix(y_true, predicted_class).ravel() # shape [ [True-Positive, False-positive], [False-negative, True-negative] ]
        ACC = (TP+TN)/(TP+TN+FP+FN)
        ACC_collecton.append(ACC)
        print(ACC)
        del model
        import torch
        import gc
        torch.cuda.memory_reserved()
        gc.collect()

import collections
model_parameters =collections.defaultdict(list)
a=0
for i in range(len(CNN_channel)):
  for j in range(len(dense_node)):
    for k in range(len(kernel_size)):
      for l in range(len(stride_size)):
        iter_num = str(a)
        model_parameters[iter_num].append(CNN_channel[i])
        model_parameters[iter_num].append(dense_node[j])
        model_parameters[iter_num].append(kernel_size[k])
        model_parameters[iter_num].append(stride_size[l])
        model_parameters[iter_num].append(ACC_collecton[a])
        a=a+1
# export the DataFrame to an Excel file
pd.DataFrame(model_parameters).to_excel('320_grid_performance output.xlsx', index=False)

### 480 feature dimension embedding test

In [ ]:
import numpy as np
import pandas as pd
import esm

# loading the y dataset for model development 
dataset = pd.read_excel('allergens_dataset.xlsx',na_filter = False) # take care the NA sequence problem
y = dataset['label']
y = np.array(y) # transformed as np.array for CNN model

# read the peptide embeddings
X_data_name = 'whole_sample_dataset_esm2_t12_35M_UR50D_unified_480_dimension.csv'
X_data = pd.read_csv(X_data_name,header=0, index_col = 0,delimiter=',')
X = np.array(X_data)

# split dataset as training and test dataset as ratio of 8:2
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.2, random_state=123)

# normalize the X data range
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train) # normalize X to 0-1 range 
X_test = scaler.transform(X_test)

ACC_collecton = []
CNN_channel = [16,32,64,128,256]
dense_node = [32,64,128,256,512,1024,2048,4096,8192]
kernel_size = [3,6,9,12]
stride_size = [1,2,4,8]
for i in range(len(CNN_channel)):
  for j in range(len(dense_node)):
    for k in range(len(kernel_size)):
      for l in range(len(stride_size)):
        inputShape=(480,1) # input feature size 
        input = Input(inputShape)
        x = Conv1D(CNN_channel[i],(kernel_size[k]),strides = (stride_size[l]),name='layer_conv2',padding='same')(input)
        x = BatchNormalization()(x)
        x = Activation('relu')(x)
        x = MaxPooling1D((2), name='MaxPool2',padding="same")(x)
        x = Dropout(0.15)(x)
        x = Flatten()(x)
        x = Dense(dense_node[j],activation = 'relu',name='fc1')(x)
        x = Dropout(0.15)(x)
        x = Dense(2,activation = 'softmax',name='fc2')(x)
        model = Model(inputs = input,outputs = x,name='Predict')
        # define SGD optimizer
        momentum = 0.5
        sgd = SGD(lr=0.01, momentum=momentum, decay=0.0, nesterov=False)
        # compile the model
        model.compile(loss='sparse_categorical_crossentropy',optimizer=sgd, metrics=['accuracy'])
        # learning deccay setting
        import math
        def step_decay(epoch): # gradually decrease the learning rate 
            initial_lrate=0.1
            drop=0.6
            epochs_drop = 3.0
            lrate= initial_lrate * math.pow(drop,    # math.pow base raised to a power
                  math.floor((1+epoch)/epochs_drop)) # math.floor Round numbers down to the nearest integer
            return lrate
        lr = LearningRateScheduler(step_decay)
        # early stop setting
        early_stop = EarlyStopping(monitor='val_accuracy', patience = 20,verbose=0,restore_best_weights = True)
        # set checkpoint and save the best model
        mc = ModelCheckpoint('best_model_grid_480.h5',  monitor='val_accuracy', mode='max', verbose=0, save_best_only=True, save_weights_only=False)
        # summary the callbacks_list
        callbacks_list = [ lr , early_stop, mc]
        model_history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=200,callbacks=callbacks_list,batch_size = 32, verbose=0)
        # load the save best model
        saved_model = load_model('best_model_grid_480.h5')
        # result collection list
        # confusion matrix 
        predicted_class= []
        predicted_protability = saved_model.predict(X_test,batch_size=1)
        for p in range(predicted_protability.shape[0]):
          index = np.where(predicted_protability[p] == np.amax(predicted_protability[p]))[0][0]
          predicted_class.append(index)
        predicted_class = np.array(predicted_class)
        y_true = y_test    
        from sklearn.metrics import confusion_matrix
        import math
        # np.ravel() return a flatten 1D array
        TP, FP, FN, TN = confusion_matrix(y_true, predicted_class).ravel() # shape [ [True-Positive, False-positive], [False-negative, True-negative] ]
        ACC = (TP+TN)/(TP+TN+FP+FN)
        ACC_collecton.append(ACC)
        print(ACC)
        del model
        import torch
        import gc
        torch.cuda.memory_reserved()
        gc.collect()

import collections
model_parameters =collections.defaultdict(list)
a=0
for i in range(len(CNN_channel)):
  for j in range(len(dense_node)):
    for k in range(len(kernel_size)):
      for l in range(len(stride_size)):
        iter_num = str(a)
        model_parameters[iter_num].append(CNN_channel[i])
        model_parameters[iter_num].append(dense_node[j])
        model_parameters[iter_num].append(kernel_size[k])
        model_parameters[iter_num].append(stride_size[l])
        model_parameters[iter_num].append(ACC_collecton[a])
        a=a+1
# export the DataFrame to an Excel file
pd.DataFrame(model_parameters).to_excel('480_grid_performance output.xlsx', index=False)

### 640 feature dimension embedding test

In [ ]:
import numpy as np
import pandas as pd
import esm

# loading the y dataset for model development 
dataset = pd.read_excel('allergens_dataset.xlsx',na_filter = False) # take care the NA sequence problem
y = dataset['label']
y = np.array(y) # transformed as np.array for CNN model

# read the peptide embeddings
X_data_name = 'whole_sample_dataset_esm2_t30_150M_UR50D_unified_640_dimension.csv'
X_data = pd.read_csv(X_data_name,header=0, index_col = 0,delimiter=',')
X = np.array(X_data)

# split dataset as training and test dataset as ratio of 8:2
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.2, random_state=123)

# normalize the X data range
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train) # normalize X to 0-1 range 
X_test = scaler.transform(X_test)

ACC_collecton = []
CNN_channel = [16,32,64,128,256]
dense_node = [32,64,128,256,512,1024,2048,4096,8192]
kernel_size = [3,6,9,12]
stride_size = [1,2,4,8]
for i in range(len(CNN_channel)):
  for j in range(len(dense_node)):
    for k in range(len(kernel_size)):
      for l in range(len(stride_size)):
        inputShape=(640,1) # input feature size 
        input = Input(inputShape)
        x = Conv1D(CNN_channel[i],(kernel_size[k]),strides = (stride_size[l]),name='layer_conv2',padding='same')(input)
        x = BatchNormalization()(x)
        x = Activation('relu')(x)
        x = MaxPooling1D((2), name='MaxPool2',padding="same")(x)
        x = Dropout(0.15)(x)
        x = Flatten()(x)
        x = Dense(dense_node[j],activation = 'relu',name='fc1')(x)
        x = Dropout(0.15)(x)
        x = Dense(2,activation = 'softmax',name='fc2')(x)
        model = Model(inputs = input,outputs = x,name='Predict')
        # define SGD optimizer
        momentum = 0.5
        sgd = SGD(lr=0.01, momentum=momentum, decay=0.0, nesterov=False)
        # compile the model
        model.compile(loss='sparse_categorical_crossentropy',optimizer=sgd, metrics=['accuracy'])
        # learning deccay setting
        import math
        def step_decay(epoch): # gradually decrease the learning rate 
            initial_lrate=0.1
            drop=0.6
            epochs_drop = 3.0
            lrate= initial_lrate * math.pow(drop,    # math.pow base raised to a power
                  math.floor((1+epoch)/epochs_drop)) # math.floor Round numbers down to the nearest integer
            return lrate
        lr = LearningRateScheduler(step_decay)
        # early stop setting
        early_stop = EarlyStopping(monitor='val_accuracy', patience = 20,verbose=0,restore_best_weights = True)
        # set checkpoint and save the best model
        mc = ModelCheckpoint('best_model_grid_640.h5',  monitor='val_accuracy', mode='max', verbose=0, save_best_only=True, save_weights_only=False)
        # summary the callbacks_list
        callbacks_list = [ lr , early_stop, mc]
        model_history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=200,callbacks=callbacks_list,batch_size = 32, verbose=0)
        # load the save best model
        saved_model = load_model('best_model_grid_640.h5')
        # result collection list
        # confusion matrix 
        predicted_class= []
        predicted_protability = saved_model.predict(X_test,batch_size=1)
        for p in range(predicted_protability.shape[0]):
          index = np.where(predicted_protability[p] == np.amax(predicted_protability[p]))[0][0]
          predicted_class.append(index)
        predicted_class = np.array(predicted_class)
        y_true = y_test    
        from sklearn.metrics import confusion_matrix
        import math
        # np.ravel() return a flatten 1D array
        TP, FP, FN, TN = confusion_matrix(y_true, predicted_class).ravel() # shape [ [True-Positive, False-positive], [False-negative, True-negative] ]
        ACC = (TP+TN)/(TP+TN+FP+FN)
        ACC_collecton.append(ACC)
        print(ACC)
        del model
        import torch
        import gc
        torch.cuda.memory_reserved()
        gc.collect()
import collections
model_parameters =collections.defaultdict(list)
a=0
for i in range(len(CNN_channel)):
  for j in range(len(dense_node)):
    for k in range(len(kernel_size)):
      for l in range(len(stride_size)):
        iter_num = str(a)
        model_parameters[iter_num].append(CNN_channel[i])
        model_parameters[iter_num].append(dense_node[j])
        model_parameters[iter_num].append(kernel_size[k])
        model_parameters[iter_num].append(stride_size[l])
        model_parameters[iter_num].append(ACC_collecton[a])
        a=a+1
# export the DataFrame to an Excel file
pd.DataFrame(model_parameters).to_excel('640_grid_performance output.xlsx', index=False)

### 1280 feature dimension embedding test


In [ ]:
import numpy as np
import pandas as pd
import esm

# loading the y dataset for model development 
dataset = pd.read_excel('allergens_dataset.xlsx',na_filter = False) # take care the NA sequence problem
y = dataset['label']
y = np.array(y) # transformed as np.array for CNN model

# read the peptide embeddings
X_data_name = 'whole_sample_dataset_esm2_t33_650M_UR50D_unified_1280_dimension.csv'
X_data = pd.read_csv(X_data_name,header=0, index_col = 0,delimiter=',')
X = np.array(X_data)

# split dataset as training and test dataset as ratio of 8:2
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.2, random_state=123)

# normalize the X data range
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train) # normalize X to 0-1 range 
X_test = scaler.transform(X_test)

ACC_collecton = []
CNN_channel = [16,32,64,128,256]
dense_node = [32,64,128,256,512,1024,2048,4096,8192]
kernel_size = [3,6,9,12]
stride_size = [1,2,4,8]
for i in range(len(CNN_channel)):
  for j in range(len(dense_node)):
    for k in range(len(kernel_size)):
      for l in range(len(stride_size)):
        inputShape=(1280,1) # input feature size 
        input = Input(inputShape)
        x = Conv1D(CNN_channel[i],(kernel_size[k]),strides = (stride_size[l]),name='layer_conv2',padding='same')(input)
        x = BatchNormalization()(x)
        x = Activation('relu')(x)
        x = MaxPooling1D((2), name='MaxPool2',padding="same")(x)
        x = Dropout(0.15)(x)
        x = Flatten()(x)
        x = Dense(dense_node[j],activation = 'relu',name='fc1')(x)
        x = Dropout(0.15)(x)
        x = Dense(2,activation = 'softmax',name='fc2')(x)
        model = Model(inputs = input,outputs = x,name='Predict')
        # define SGD optimizer
        momentum = 0.5
        sgd = SGD(lr=0.01, momentum=momentum, decay=0.0, nesterov=False)
        # compile the model
        model.compile(loss='sparse_categorical_crossentropy',optimizer=sgd, metrics=['accuracy'])
        # learning deccay setting
        import math
        def step_decay(epoch): # gradually decrease the learning rate 
            initial_lrate=0.1
            drop=0.6
            epochs_drop = 3.0
            lrate= initial_lrate * math.pow(drop,    # math.pow base raised to a power
                  math.floor((1+epoch)/epochs_drop)) # math.floor Round numbers down to the nearest integer
            return lrate
        lr = LearningRateScheduler(step_decay)
        # early stop setting
        early_stop = EarlyStopping(monitor='val_accuracy', patience = 20,verbose=0,restore_best_weights = True)
        # set checkpoint and save the best model
        mc = ModelCheckpoint('best_model_grid_1280.h5',  monitor='val_accuracy', mode='max', verbose=0, save_best_only=True, save_weights_only=False)
        # summary the callbacks_list
        callbacks_list = [ lr , early_stop, mc]
        model_history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=200,callbacks=callbacks_list,batch_size = 32, verbose=0)
        # load the save best model
        saved_model = load_model('best_model_grid_1280.h5')
        # result collection list
        # confusion matrix 
        predicted_class= []
        predicted_protability = saved_model.predict(X_test,batch_size=1)
        for p in range(predicted_protability.shape[0]):
          index = np.where(predicted_protability[p] == np.amax(predicted_protability[p]))[0][0]
          predicted_class.append(index)
        predicted_class = np.array(predicted_class)
        y_true = y_test    
        from sklearn.metrics import confusion_matrix
        import math
        # np.ravel() return a flatten 1D array
        TP, FP, FN, TN = confusion_matrix(y_true, predicted_class).ravel() # shape [ [True-Positive, False-positive], [False-negative, True-negative] ]
        ACC = (TP+TN)/(TP+TN+FP+FN)
        ACC_collecton.append(ACC)
        print(ACC)
        del model
        import torch
        import gc
        torch.cuda.memory_reserved()
        gc.collect()
import collections
model_parameters =collections.defaultdict(list)
a=0
for i in range(len(CNN_channel)):
  for j in range(len(dense_node)):
    for k in range(len(kernel_size)):
      for l in range(len(stride_size)):
        iter_num = str(a)
        model_parameters[iter_num].append(CNN_channel[i])
        model_parameters[iter_num].append(dense_node[j])
        model_parameters[iter_num].append(kernel_size[k])
        model_parameters[iter_num].append(stride_size[l])
        model_parameters[iter_num].append(ACC_collecton[a])
        a=a+1
# export the DataFrame to an Excel file
pd.DataFrame(model_parameters).to_excel('1280_grid_performance output.xlsx', index=False)

### 2560 feature dimension embedding test
Due to the enourmous computational needs and also the limitted performance improivement, we do not test the 128 and 256 fileter size with 8196 units.

In [ ]:
import numpy as np
import pandas as pd
import esm

# loading the y dataset for model development 
dataset = pd.read_excel('allergens_dataset.xlsx',na_filter = False) # take care the NA sequence problem
y = dataset['label']
y = np.array(y) # transformed as np.array for CNN model

# read the peptide embeddings
X_data_name = 'whole_sample_dataset_esm2_t36_3B_UR50D_unified_2560_dimension.csv'
X_data = pd.read_csv(X_data_name,header=0, index_col = 0,delimiter=',')
X = np.array(X_data)

# split dataset as training and test dataset as ratio of 8:2
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.2, random_state=123)

# normalize the X data range
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train) # normalize X to 0-1 range 
X_test = scaler.transform(X_test)

ACC_collecton = []
CNN_channel = [16,32,64,128,256] # 
dense_node = [32,64,128,256,512,1024,2048,4096]
kernel_size = [3,6,9,12]
stride_size = [1,2,4,8]
for i in range(len(CNN_channel)):
  for j in range(len(dense_node)):
    for k in range(len(kernel_size)):
      for l in range(len(stride_size)):
        inputShape=(2560,1) # input feature size 
        input = Input(inputShape)
        x = Conv1D(CNN_channel[i],(kernel_size[k]),strides = (stride_size[l]),name='layer_conv2',padding='same')(input)
        x = BatchNormalization()(x)
        x = Activation('relu')(x)
        x = MaxPooling1D((2), name='MaxPool2',padding="same")(x)
        x = Dropout(0.15)(x)
        x = Flatten()(x)
        x = Dense(dense_node[j],activation = 'relu',name='fc1')(x)
        x = Dropout(0.15)(x)
        x = Dense(2,activation = 'softmax',name='fc2')(x)
        model = Model(inputs = input,outputs = x,name='Predict')
        # define SGD optimizer
        momentum = 0.5
        sgd = SGD(lr=0.01, momentum=momentum, decay=0.0, nesterov=False)
        # compile the model
        model.compile(loss='sparse_categorical_crossentropy',optimizer=sgd, metrics=['accuracy'])
        # learning deccay setting
        import math
        def step_decay(epoch): # gradually decrease the learning rate 
            initial_lrate=0.1
            drop=0.6
            epochs_drop = 3.0
            lrate= initial_lrate * math.pow(drop,    # math.pow base raised to a power
                  math.floor((1+epoch)/epochs_drop)) # math.floor Round numbers down to the nearest integer
            return lrate
        lr = LearningRateScheduler(step_decay)
        # early stop setting
        early_stop = EarlyStopping(monitor='val_accuracy', patience = 20,verbose=0,restore_best_weights = True)
        # set checkpoint and save the best model
        mc = ModelCheckpoint('best_model_grid_2560.h5',  monitor='val_accuracy', mode='max', verbose=0, save_best_only=True, save_weights_only=False)
        # summary the callbacks_list
        callbacks_list = [ lr , early_stop, mc]
        model_history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=200,callbacks=callbacks_list,batch_size = 32, verbose=0)
        # load the save best model
        saved_model = load_model('best_model_grid_2560.h5')
        # result collection list
        # confusion matrix 
        predicted_class= []
        predicted_protability = saved_model.predict(X_test,batch_size=1)
        for p in range(predicted_protability.shape[0]):
          index = np.where(predicted_protability[p] == np.amax(predicted_protability[p]))[0][0]
          predicted_class.append(index)
        predicted_class = np.array(predicted_class)
        y_true = y_test    
        from sklearn.metrics import confusion_matrix
        import math
        # np.ravel() return a flatten 1D array
        TP, FP, FN, TN = confusion_matrix(y_true, predicted_class).ravel() # shape [ [True-Positive, False-positive], [False-negative, True-negative] ]
        ACC = (TP+TN)/(TP+TN+FP+FN)
        ACC_collecton.append(ACC)
        print(ACC)
        del model
        import torch
        import gc
        torch.cuda.memory_reserved()
        gc.collect()
import collections
model_parameters =collections.defaultdict(list)
a=0
for i in range(len(CNN_channel)):
  for j in range(len(dense_node)):
    for k in range(len(kernel_size)):
      for l in range(len(stride_size)):
        iter_num = str(a)
        model_parameters[iter_num].append(CNN_channel[i])
        model_parameters[iter_num].append(dense_node[j])
        model_parameters[iter_num].append(kernel_size[k])
        model_parameters[iter_num].append(stride_size[l])
        model_parameters[iter_num].append(ACC_collecton[a])
        a=a+1
# export the DataFrame to an Excel file
pd.DataFrame(model_parameters).to_excel('2560_grid_performance output.xlsx', index=False)

## 10 times random dataset division test

### 320 feature dimension embedding test

In [ ]:
# set the best parameters
CNN_channel = [32, 64, 16, 32, 32, 16, 16, 32, 32, 32, 128, 64, 32, 32, 64, 64, 128, 32, 64, 16, 128, 32, 16, 32, 32, 32]
dense_node = [32, 32, 64, 64, 128, 128, 256, 256, 512, 512, 8192, 4096, 8192, 4096, 4096, 4096, 4096, 4096, 8192, 1024, 2048, 2048, 2048, 1024, 512, 512]
kernel_size = [9, 6, 1, 6, 12, 12, 12, 3, 3, 9, 3, 12, 9, 6, 12, 12, 12, 12, 12, 9, 12, 12, 6, 6, 3, 12]
stride_size = [1, 4, 2, 1, 1, 1, 2, 1, 1, 1, 1, 2, 2, 1, 1, 4, 4, 1, 2, 1, 8, 1, 1, 1, 1, 2]
print(len(CNN_channel),len(dense_node), len(kernel_size),len(stride_size))

26 26 26 26


In [ ]:
import numpy as np
import pandas as pd
import esm
# whole dataset loading and dataset splitting 
dataset = pd.read_excel('allergens_dataset.xlsx',na_filter = False) # take care the NA sequence problem

# generate the peptide embeddings
sequence_list = dataset['sequence'] 
y = dataset['label']
y = np.array(y) # transformed as np.array for CNN model

# read the peptide embeddings
X_data_name = 'whole_sample_dataset_esm2_t6_8M_UR50D_unified_320_dimension.csv'
X_data = pd.read_csv(X_data_name,header=0, index_col = 0,delimiter=',')
X = np.array(X_data)

ACC_repeat_col=[]
BACC_repeat_col = []
Sn_repeat_col= []
Sp_repeat_col=[]
MCC_repeat_col=[]
AUC_repeat_col=[]

for i in range(len(CNN_channel)):
  # split dataset as training and test dataset as ratio of 8:2
  ACC_collecton = []
  BACC_collecton = []
  Sn_collecton = []
  Sp_collecton = []
  MCC_collecton = []
  AUC_collecton = []
  for random_num in range(10):
    from sklearn.model_selection import train_test_split
    X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.2, random_state=random_num)

    # normalize the X data range
    from sklearn.preprocessing import MinMaxScaler
    scaler = MinMaxScaler()
    scaler.fit(X_train)
    X_train = scaler.transform(X_train) # normalize X to 0-1 range 
    X_test = scaler.transform(X_test)

    inputShape=(320,1) # input feature size 
    input = Input(inputShape)
    x = Conv1D(CNN_channel[i],(kernel_size[i]),strides = (stride_size[i]),name='layer_conv2',padding='same')(input)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = MaxPooling1D((2), name='MaxPool2',padding="same")(x)
    x = Dropout(0.15)(x)
    x = Flatten()(x)
    x = Dense(dense_node[i],activation = 'relu',name='fc1')(x)
    x = Dropout(0.15)(x)
    x = Dense(2,activation = 'softmax',name='fc2')(x)
    model = Model(inputs = input,outputs = x,name='Predict')
    # define SGD optimizer
    momentum = 0.5
    sgd = SGD(lr=0.01, momentum=momentum, decay=0.0, nesterov=False)
    # compile the model
    model.compile(loss='sparse_categorical_crossentropy',optimizer=sgd, metrics=['accuracy'])
    # learning deccay setting
    import math
    def step_decay(epoch): # gradually decrease the learning rate 
        initial_lrate=0.1
        drop=0.6
        epochs_drop = 3.0
        lrate= initial_lrate * math.pow(drop,    # math.pow base raised to a power
              math.floor((1+epoch)/epochs_drop)) # math.floor Round numbers down to the nearest integer
        return lrate
    lr = LearningRateScheduler(step_decay)
    # early stop setting
    early_stop = EarlyStopping(monitor='val_accuracy', patience = 40,verbose=0,restore_best_weights = True)
    # set checkpoint and save the best model
    mc = ModelCheckpoint('best_modelbest_320_10times.h5',  monitor='val_accuracy', mode='max', verbose=0, save_best_only=True, save_weights_only=False)
    # summary the callbacks_list
    callbacks_list = [ lr , early_stop, mc]
    model_history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=200,callbacks=callbacks_list,batch_size = 16, verbose=0)
    # load the save best model
    saved_model = load_model('best_modelbest_320_10times.h5')
    # result collection list
    # confusion matrix 
    predicted_class= []
    predicted_protability = saved_model.predict(X_test,batch_size=1)
    for p in range(predicted_protability.shape[0]):
      index = np.where(predicted_protability[p] == np.amax(predicted_protability[p]))[0][0]
      predicted_class.append(index)
    predicted_class = np.array(predicted_class)
    y_true = y_test    
    from sklearn.metrics import confusion_matrix
    import math
    # np.ravel() return a flatten 1D array
    TP, FP, FN, TN = confusion_matrix(y_true, predicted_class).ravel() # shape [ [True-Positive, False-positive], [False-negative, True-negative] ]
    ACC = (TP+TN)/(TP+TN+FP+FN)
    ACC_collecton.append(ACC)
    Sn_collecton.append(TP/(TP+FN))
    Sp_collecton.append(TN/(TN+FP))
    MCC = (TP*TN-FP*FN)/math.pow(((TP+FP)*(TP+FN)*(TN+FP)*(TN+FN)),0.5)
    MCC_collecton.append(MCC)
    BACC_collecton.append(0.5*TP/(TP+FN)+0.5*TN/(TN+FP))
    from sklearn.metrics import roc_auc_score
    AUC = roc_auc_score(y_test, predicted_protability[:,1])
    AUC_collecton.append(AUC)
    del model
    import torch
    import gc
    torch.cuda.memory_reserved()
    gc.collect()
  import statistics
  ACC_repeat_col.append(str(round(statistics.mean(ACC_collecton),3))+'±'+ str(round(statistics.stdev(ACC_collecton),3)))
  BACC_repeat_col.append(str(round(statistics.mean(BACC_collecton),3))+'±'+str(round(statistics.stdev(BACC_collecton),3)))
  Sn_repeat_col.append(str(round(statistics.mean(Sn_collecton),3))+'±'+str(round(statistics.stdev(Sn_collecton),3)))
  Sp_repeat_col.append(str(round(statistics.mean(Sp_collecton),3))+'±'+str(round(statistics.stdev(Sp_collecton),3)))
  MCC_repeat_col.append(str(round(statistics.mean(MCC_collecton),3))+'±'+str(round(statistics.stdev(MCC_collecton),3)))
  AUC_repeat_col.append(str(round(statistics.mean(AUC_collecton),3))+'±'+str(round(statistics.stdev(AUC_collecton),3)))

In [ ]:
# combine the lists into a list of tuples
combined_list = list(zip(ACC_repeat_col, BACC_repeat_col, Sn_repeat_col, Sp_repeat_col, MCC_repeat_col, AUC_repeat_col))

# create a DataFrame from the list of tuples
df = pd.DataFrame(combined_list, columns=['ACC_collection', 'BACC_collection', 'Sn_collection', 'Sp_collection', 'MCC_collection', 'AUC_collection'])

# export the DataFrame to an Excel file
df.to_excel('320 ten times repeat performance output.xlsx', index=False)

### 480 feature dimension embedding test

In [ ]:
# set as the best parameters
CNN_channel = [128, 32, 64, 64, 32, 16, 16, 16, 16, 16, 64, 64, 64, 16, 128, 16, 32, 32, 16, 32, 32, 64, 32, 32, 16, 16, 32, 32, 16, 32]
dense_node = [8192, 4096, 8192, 8192, 4096, 8192, 8192, 2048, 8192, 2048, 2048, 2048, 1024, 2048, 1024, 2048, 512, 1024, 512, 32, 32, 64, 64, 128, 128, 256, 256, 512, 512, 512]
kernel_size = [9, 6, 6, 9, 9, 12, 6, 12, 9, 9, 6, 12, 12, 6, 12, 9, 6, 12, 12, 6, 12, 6, 9, 9, 12, 6, 9, 6, 12, 6]
stride_size = [8, 1, 4, 8, 4, 2, 2, 2, 2, 2, 8, 4, 4, 1, 4, 1, 4, 1, 1, 1, 1, 2, 4, 1, 1, 2, 1, 4, 1, 2]
print(len(CNN_channel),len(dense_node), len(kernel_size),len(stride_size))

30 30 30 30


In [ ]:
import numpy as np
import pandas as pd
import esm

# whole dataset loading and dataset splitting 
dataset = pd.read_excel('allergens_dataset.xlsx',na_filter = False) # take care the NA sequence problem
# # generate the peptide embeddings
sequence_list = dataset['sequence'] 

# loading the y dataset for model development 
y = dataset['label']
y = np.array(y) # transformed as np.array for CNN model

# read the peptide embeddings
X_data_name = 'whole_sample_dataset_esm2_t12_35M_UR50D_unified_480_dimension.csv'
X_data = pd.read_csv(X_data_name,header=0, index_col = 0,delimiter=',')
X = np.array(X_data)

ACC_repeat_col=[]
BACC_repeat_col = []
Sn_repeat_col= []
Sp_repeat_col=[]
MCC_repeat_col=[]
AUC_repeat_col=[]
for i in range(len(CNN_channel)):
  # split dataset as training and test dataset as ratio of 8:2
  ACC_collecton = []
  BACC_collecton = []
  Sn_collecton = []
  Sp_collecton = []
  MCC_collecton = []
  AUC_collecton = []
  for random_num in range(10):
    from sklearn.model_selection import train_test_split
    X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.2, random_state=random_num)

    # normalize the X data range
    from sklearn.preprocessing import MinMaxScaler
    scaler = MinMaxScaler()
    scaler.fit(X_train)
    X_train = scaler.transform(X_train) # normalize X to 0-1 range 
    X_test = scaler.transform(X_test)

    inputShape=(480,1) # input feature size 
    input = Input(inputShape)
    x = Conv1D(CNN_channel[i],(kernel_size[i]),strides = (stride_size[i]),name='layer_conv2',padding='same')(input)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = MaxPooling1D((2), name='MaxPool2',padding="same")(x)
    x = Dropout(0.15)(x)
    x = Flatten()(x)
    x = Dense(dense_node[i],activation = 'relu',name='fc1')(x)
    x = Dropout(0.15)(x)
    x = Dense(2,activation = 'softmax',name='fc2')(x)
    model = Model(inputs = input,outputs = x,name='Predict')
    # define SGD optimizer
    momentum = 0.5
    sgd = SGD(lr=0.01, momentum=momentum, decay=0.0, nesterov=False)
    # compile the model
    model.compile(loss='sparse_categorical_crossentropy',optimizer=sgd, metrics=['accuracy'])
    # learning deccay setting
    import math
    def step_decay(epoch): # gradually decrease the learning rate 
        initial_lrate=0.1
        drop=0.6
        epochs_drop = 3.0
        lrate= initial_lrate * math.pow(drop,    # math.pow base raised to a power
              math.floor((1+epoch)/epochs_drop)) # math.floor Round numbers down to the nearest integer
        return lrate
    lr = LearningRateScheduler(step_decay)
    # early stop setting
    early_stop = EarlyStopping(monitor='val_accuracy', patience = 40,verbose=0,restore_best_weights = True)
    # set checkpoint and save the best model
    mc = ModelCheckpoint('best_modelbest_480_repeat_1.h5',  monitor='val_accuracy', mode='max', verbose=0, save_best_only=True, save_weights_only=False)
    # summary the callbacks_list
    callbacks_list = [ lr , early_stop, mc]
    model_history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=200,callbacks=callbacks_list,batch_size = 16, verbose=0)
    # load the save best model
    saved_model = load_model('best_modelbest_480_repeat_1.h5')
    # result collection list
    # confusion matrix 
    predicted_class= []
    predicted_protability = saved_model.predict(X_test,batch_size=1)
    for p in range(predicted_protability.shape[0]):
      index = np.where(predicted_protability[p] == np.amax(predicted_protability[p]))[0][0]
      predicted_class.append(index)
    predicted_class = np.array(predicted_class)
    y_true = y_test    
    from sklearn.metrics import confusion_matrix
    import math
    # np.ravel() return a flatten 1D array
    TP, FP, FN, TN = confusion_matrix(y_true, predicted_class).ravel() # shape [ [True-Positive, False-positive], [False-negative, True-negative] ]
    ACC = (TP+TN)/(TP+TN+FP+FN)
    ACC_collecton.append(ACC)
    Sn_collecton.append(TP/(TP+FN))
    Sp_collecton.append(TN/(TN+FP))
    MCC = (TP*TN-FP*FN)/math.pow(((TP+FP)*(TP+FN)*(TN+FP)*(TN+FN)),0.5)
    MCC_collecton.append(MCC)
    BACC_collecton.append(0.5*TP/(TP+FN)+0.5*TN/(TN+FP))
    from sklearn.metrics import roc_auc_score
    AUC = roc_auc_score(y_test, predicted_protability[:,1])
    AUC_collecton.append(AUC)
    del model
    import torch
    import gc
    torch.cuda.memory_reserved()
    gc.collect()
  import statistics
  ACC_repeat_col.append(str(round(statistics.mean(ACC_collecton),3))+'±'+ str(round(statistics.stdev(ACC_collecton),3)))
  BACC_repeat_col.append(str(round(statistics.mean(BACC_collecton),3))+'±'+str(round(statistics.stdev(BACC_collecton),3)))
  Sn_repeat_col.append(str(round(statistics.mean(Sn_collecton),3))+'±'+str(round(statistics.stdev(Sn_collecton),3)))
  Sp_repeat_col.append(str(round(statistics.mean(Sp_collecton),3))+'±'+str(round(statistics.stdev(Sp_collecton),3)))
  MCC_repeat_col.append(str(round(statistics.mean(MCC_collecton),3))+'±'+str(round(statistics.stdev(MCC_collecton),3)))
  AUC_repeat_col.append(str(round(statistics.mean(AUC_collecton),3))+'±'+str(round(statistics.stdev(AUC_collecton),3)))
  print(ACC_repeat_col[i],BACC_repeat_col[i],Sn_repeat_col[i],Sp_repeat_col[i],MCC_repeat_col[i],AUC_repeat_col[i])

In [ ]:
# combine the lists into a list of tuples
combined_list = list(zip(ACC_repeat_col, BACC_repeat_col, Sn_repeat_col, Sp_repeat_col, MCC_repeat_col, AUC_repeat_col))

# create a DataFrame from the list of tuples
df = pd.DataFrame(combined_list, columns=['ACC_collection', 'BACC_collection', 'Sn_collection', 'Sp_collection', 'MCC_collection', 'AUC_collection'])

# export the DataFrame to an Excel file
df.to_excel('480 ten times repeat performance output.xlsx', index=False)

### 640 feature dimension embedding test

In [ ]:
# set the best parameters
CNN_channel = [64, 64, 64, 32, 64, 16, 128, 64, 32, 32, 32, 64, 32, 16, 128, 16, 128, 64, 64, 64, 16, 64, 64, 16, 32, 128, 32, 16, 32, 64, 32, 64, 64, 64]
dense_node = [8192, 2048, 8192, 8192, 4096, 2048, 4096, 2048, 2048, 4096, 512, 1024, 4096, 2048, 4096, 1024, 512, 512, 1024, 512, 128, 1024, 512, 32, 32, 64, 64, 128, 128, 256, 256, 512, 512, 512]
kernel_size = [12, 12, 6, 9, 12, 12, 6, 9, 9, 9, 9, 9, 12, 9, 9, 12, 9, 9, 9, 12, 9, 6, 9, 12, 9, 12, 9, 9, 9, 9, 6, 9, 12, 9]
stride_size = [2, 4, 4, 2, 2, 2, 4, 2, 1, 2, 1, 2, 2, 2, 2, 2, 1, 1, 1, 2, 1, 4, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2]
print(len(CNN_channel),len(dense_node), len(kernel_size),len(stride_size))

34 34 34 34


In [ ]:
import numpy as np
import pandas as pd
import esm

# whole dataset loading and dataset splitting 
dataset = pd.read_excel('allergens_dataset.xlsx',na_filter = False) # take care the NA sequence problem
# generate the peptide embeddings
sequence_list = dataset['sequence'] 
# loading the y dataset for model development 
y = dataset['label']
y = np.array(y) # transformed as np.array for CNN model

# read the peptide embeddings
X_data_name = 'whole_sample_dataset_esm2_t30_150M_UR50D_unified_640_dimension.csv'
X_data = pd.read_csv(X_data_name,header=0, index_col = 0,delimiter=',')
X = np.array(X_data)

ACC_repeat_col=[]
BACC_repeat_col = []
Sn_repeat_col= []
Sp_repeat_col=[]
MCC_repeat_col=[]
AUC_repeat_col=[]

for i in range(len(CNN_channel)):
  # split dataset as training and test dataset as ratio of 8:2
  ACC_collecton = []
  BACC_collecton = []
  Sn_collecton = []
  Sp_collecton = []
  MCC_collecton = []
  AUC_collecton = []
  for random_num in range(10):
    from sklearn.model_selection import train_test_split
    X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.2, random_state=random_num)

    # normalize the X data range
    from sklearn.preprocessing import MinMaxScaler
    scaler = MinMaxScaler()
    scaler.fit(X_train)
    X_train = scaler.transform(X_train) # normalize X to 0-1 range 
    X_test = scaler.transform(X_test)

    inputShape=(640,1) # input feature size 
    input = Input(inputShape)
    x = Conv1D(CNN_channel[i],(kernel_size[i]),strides = (stride_size[i]),name='layer_conv2',padding='same')(input)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = MaxPooling1D((2), name='MaxPool2',padding="same")(x)
    x = Dropout(0.15)(x)
    x = Flatten()(x)
    x = Dense(dense_node[i],activation = 'relu',name='fc1')(x)
    x = Dropout(0.15)(x)
    x = Dense(2,activation = 'softmax',name='fc2')(x)
    model = Model(inputs = input,outputs = x,name='Predict')
    # define SGD optimizer
    momentum = 0.5
    sgd = SGD(lr=0.01, momentum=momentum, decay=0.0, nesterov=False)
    # compile the model
    model.compile(loss='sparse_categorical_crossentropy',optimizer=sgd, metrics=['accuracy'])
    # learning deccay setting
    import math
    def step_decay(epoch): # gradually decrease the learning rate 
        initial_lrate=0.1
        drop=0.6
        epochs_drop = 3.0
        lrate= initial_lrate * math.pow(drop,    # math.pow base raised to a power
              math.floor((1+epoch)/epochs_drop)) # math.floor Round numbers down to the nearest integer
        return lrate
    lr = LearningRateScheduler(step_decay)
    # early stop setting
    early_stop = EarlyStopping(monitor='val_accuracy', patience = 40,verbose=0,restore_best_weights = True)
    # set checkpoint and save the best model
    mc = ModelCheckpoint('best_modelbest_640_repeat_10.h5',  monitor='val_accuracy', mode='max', verbose=0, save_best_only=True, save_weights_only=False)
    # summary the callbacks_list
    callbacks_list = [ lr , early_stop, mc]
    model_history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=200,callbacks=callbacks_list,batch_size = 16, verbose=0)
    # load the save best model
    saved_model = load_model('best_modelbest_640_repeat_10.h5')
    # result collection list
    # confusion matrix 
    predicted_class= []
    predicted_protability = saved_model.predict(X_test,batch_size=1)
    for p in range(predicted_protability.shape[0]):
      index = np.where(predicted_protability[p] == np.amax(predicted_protability[p]))[0][0]
      predicted_class.append(index)
    predicted_class = np.array(predicted_class)
    y_true = y_test    
    from sklearn.metrics import confusion_matrix
    import math
    # np.ravel() return a flatten 1D array
    TP, FP, FN, TN = confusion_matrix(y_true, predicted_class).ravel() # shape [ [True-Positive, False-positive], [False-negative, True-negative] ]
    ACC = (TP+TN)/(TP+TN+FP+FN)
    ACC_collecton.append(ACC)
    Sn_collecton.append(TP/(TP+FN))
    Sp_collecton.append(TN/(TN+FP))
    MCC = (TP*TN-FP*FN)/math.pow(((TP+FP)*(TP+FN)*(TN+FP)*(TN+FN)),0.5)
    MCC_collecton.append(MCC)
    BACC_collecton.append(0.5*TP/(TP+FN)+0.5*TN/(TN+FP))
    from sklearn.metrics import roc_auc_score
    AUC = roc_auc_score(y_test, predicted_protability[:,1])
    AUC_collecton.append(AUC)
    del model
    import torch
    import gc
    torch.cuda.memory_reserved()
    gc.collect()
  import statistics
  ACC_repeat_col.append(str(round(statistics.mean(ACC_collecton),3))+'±'+ str(round(statistics.stdev(ACC_collecton),3)))
  BACC_repeat_col.append(str(round(statistics.mean(BACC_collecton),3))+'±'+str(round(statistics.stdev(BACC_collecton),3)))
  Sn_repeat_col.append(str(round(statistics.mean(Sn_collecton),3))+'±'+str(round(statistics.stdev(Sn_collecton),3)))
  Sp_repeat_col.append(str(round(statistics.mean(Sp_collecton),3))+'±'+str(round(statistics.stdev(Sp_collecton),3)))
  MCC_repeat_col.append(str(round(statistics.mean(MCC_collecton),3))+'±'+str(round(statistics.stdev(MCC_collecton),3)))
  AUC_repeat_col.append(str(round(statistics.mean(AUC_collecton),3))+'±'+str(round(statistics.stdev(AUC_collecton),3)))
  print(ACC_repeat_col[i],BACC_repeat_col[i],Sn_repeat_col[i],Sp_repeat_col[i],MCC_repeat_col[i],AUC_repeat_col[i])

In [ ]:
# combine the lists into a list of tuples
combined_list = list(zip(ACC_repeat_col, BACC_repeat_col, Sn_repeat_col, Sp_repeat_col, MCC_repeat_col, AUC_repeat_col))

# create a DataFrame from the list of tuples
df = pd.DataFrame(combined_list, columns=['ACC_collection', 'BACC_collection', 'Sn_collection', 'Sp_collection', 'MCC_collection', 'AUC_collection'])

# export the DataFrame to an Excel file
df.to_excel('640 ten times repeat performance output.xlsx', index=False)

### 1280 feature dimension embedding test

In [ ]:
# set as the best parameters
CNN_channel = [32, 128, 32, 32, 32, 16, 64, 16, 16, 64, 64, 64, 43, 128, 64, 16, 128, 64, 128, 16, 128, 16, 64, 16, 16, 256, 32, 16, 32, 128, 16, 32, 32, 64, 256, 128, 128, 16, 64, 32, 128, 256]
dense_node = [8192, 4096, 1024, 8192, 2048, 1024, 4096, 512, 8192, 2048, 512, 256, 512, 1024, 1024, 2048, 4096, 1024, 512, 128, 1024, 1024, 512, 256, 32, 32, 32, 64, 64, 64, 128, 128, 128, 256, 256, 256, 256, 512, 512, 512, 512, 256]
kernel_size = [9, 12, 9, 9, 6, 12, 9, 12, 12, 9, 9, 12, 6, 12, 9, 12, 12, 12, 12, 12, 12, 6, 12, 3, 12, 6, 12, 6, 12, 12, 12, 12, 12, 12, 9, 9, 9, 12, 9, 6, 12, 9]
stride_size = [2, 8, 8, 8, 1, 2, 2, 1, 2, 2, 8, 2, 8, 1, 2, 2, 2, 4, 8, 1, 2, 4, 2, 1, 1, 1, 1, 1, 2, 1, 1, 1, 2, 2, 1, 8, 2, 1, 8, 8, 8, 1]
print(len(CNN_channel),len(dense_node), len(kernel_size),len(stride_size))

42 42 42 42


In [ ]:
import numpy as np
import pandas as pd
import esm
# whole dataset loading and dataset splitting 
dataset = pd.read_excel('allergens_dataset.xlsx',na_filter = False) # take care the NA sequence problem
# # generate the peptide embeddings
sequence_list = dataset['sequence'] 

# loading the y dataset for model development 
y = dataset['label']
y = np.array(y) # transformed as np.array for CNN model

# read the peptide embeddings
X_data_name = 'whole_sample_dataset_esm2_t33_650M_UR50D_unified_1280_dimension.csv'
X_data = pd.read_csv(X_data_name,header=0, index_col = 0,delimiter=',')
X = np.array(X_data)

ACC_repeat_col=[]
BACC_repeat_col = []
Sn_repeat_col= []
Sp_repeat_col=[]
MCC_repeat_col=[]
AUC_repeat_col=[]

for i in range(len(CNN_channel)):
  # split dataset as training and test dataset as ratio of 8:2
  ACC_collecton = []
  BACC_collecton = []
  Sn_collecton = []
  Sp_collecton = []
  MCC_collecton = []
  AUC_collecton = []
  for random_num in range(10):
    from sklearn.model_selection import train_test_split
    X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.2, random_state=random_num)

    # normalize the X data range
    from sklearn.preprocessing import MinMaxScaler
    scaler = MinMaxScaler()
    scaler.fit(X_train)
    X_train = scaler.transform(X_train) # normalize X to 0-1 range 
    X_test = scaler.transform(X_test)

    inputShape=(1280,1) # input feature size 
    input = Input(inputShape)
    x = Conv1D(CNN_channel[i],(kernel_size[i]),strides = (stride_size[i]),name='layer_conv2',padding='same')(input)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = MaxPooling1D((2), name='MaxPool2',padding="same")(x)
    x = Dropout(0.15)(x)
    x = Flatten()(x)
    x = Dense(dense_node[i],activation = 'relu',name='fc1')(x)
    x = Dropout(0.15)(x)
    x = Dense(2,activation = 'softmax',name='fc2')(x)
    model = Model(inputs = input,outputs = x,name='Predict')
    # define SGD optimizer
    momentum = 0.5
    sgd = SGD(lr=0.01, momentum=momentum, decay=0.0, nesterov=False)
    # compile the model
    model.compile(loss='sparse_categorical_crossentropy',optimizer=sgd, metrics=['accuracy'])
    # learning deccay setting
    import math
    def step_decay(epoch): # gradually decrease the learning rate 
        initial_lrate=0.1
        drop=0.6
        epochs_drop = 3.0
        lrate= initial_lrate * math.pow(drop,    # math.pow base raised to a power
              math.floor((1+epoch)/epochs_drop)) # math.floor Round numbers down to the nearest integer
        return lrate
    lr = LearningRateScheduler(step_decay)
    # early stop setting
    early_stop = EarlyStopping(monitor='val_accuracy', patience = 40,verbose=0,restore_best_weights = True)
    # set checkpoint and save the best model
    mc = ModelCheckpoint('best_modelbest_1280_repeat_10.h5',  monitor='val_accuracy', mode='max', verbose=0, save_best_only=True, save_weights_only=False)
    # summary the callbacks_list
    callbacks_list = [ lr , early_stop, mc]
    model_history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=200,callbacks=callbacks_list,batch_size = 16, verbose=0)
    # load the save best model
    saved_model = load_model('best_modelbest_1280_repeat_10.h5')
    # result collection list
    # confusion matrix 
    predicted_class= []
    predicted_protability = saved_model.predict(X_test,batch_size=1)
    for p in range(predicted_protability.shape[0]):
      index = np.where(predicted_protability[p] == np.amax(predicted_protability[p]))[0][0]
      predicted_class.append(index)
    predicted_class = np.array(predicted_class)
    y_true = y_test    
    from sklearn.metrics import confusion_matrix
    import math
    # np.ravel() return a flatten 1D array
    TP, FP, FN, TN = confusion_matrix(y_true, predicted_class).ravel() # shape [ [True-Positive, False-positive], [False-negative, True-negative] ]
    ACC = (TP+TN)/(TP+TN+FP+FN)
    ACC_collecton.append(ACC)
    Sn_collecton.append(TP/(TP+FN))
    Sp_collecton.append(TN/(TN+FP))
    MCC = (TP*TN-FP*FN)/math.pow(((TP+FP)*(TP+FN)*(TN+FP)*(TN+FN)),0.5)
    MCC_collecton.append(MCC)
    BACC_collecton.append(0.5*TP/(TP+FN)+0.5*TN/(TN+FP))
    from sklearn.metrics import roc_auc_score
    AUC = roc_auc_score(y_test, predicted_protability[:,1])
    AUC_collecton.append(AUC)
    del model
    import torch
    import gc
    torch.cuda.memory_reserved()
    gc.collect()
  import statistics
  ACC_repeat_col.append(str(round(statistics.mean(ACC_collecton),3))+'±'+ str(round(statistics.stdev(ACC_collecton),3)))
  BACC_repeat_col.append(str(round(statistics.mean(BACC_collecton),3))+'±'+str(round(statistics.stdev(BACC_collecton),3)))
  Sn_repeat_col.append(str(round(statistics.mean(Sn_collecton),3))+'±'+str(round(statistics.stdev(Sn_collecton),3)))
  Sp_repeat_col.append(str(round(statistics.mean(Sp_collecton),3))+'±'+str(round(statistics.stdev(Sp_collecton),3)))
  MCC_repeat_col.append(str(round(statistics.mean(MCC_collecton),3))+'±'+str(round(statistics.stdev(MCC_collecton),3)))
  AUC_repeat_col.append(str(round(statistics.mean(AUC_collecton),3))+'±'+str(round(statistics.stdev(AUC_collecton),3)))
  print(ACC_repeat_col[i],BACC_repeat_col[i],Sn_repeat_col[i],Sp_repeat_col[i],MCC_repeat_col[i],AUC_repeat_col[i])

In [ ]:
# combine the lists into a list of tuples
combined_list = list(zip(ACC_repeat_col, BACC_repeat_col, Sn_repeat_col, Sp_repeat_col, MCC_repeat_col, AUC_repeat_col))

# create a DataFrame from the list of tuples
df = pd.DataFrame(combined_list, columns=['ACC_collection', 'BACC_collection', 'Sn_collection', 'Sp_collection', 'MCC_collection', 'AUC_collection'])

# export the DataFrame to an Excel file
df.to_excel('1280 10 times repeat performance output.xlsx', index=False)

### 2560 feature dimension embedding test

In [ ]:
# set best parameters
CNN_channel = [128, 64, 64, 16, 64, 128, 32, 128, 64, 32, 32, 128, 64, 16, 32, 16, 32, 64, 16, 64, 32, 32, 16, 64, 32, 32, 64, 128, 256, 32, 32, 16, 16, 16, 32, 16, 16]
dense_node = [2048, 2048, 4096, 512, 4096, 1024, 2048, 4096, 1024, 256, 2048, 1024, 128, 4096, 4096, 32, 32, 32, 32, 64, 64, 64, 64, 128, 128, 128, 128, 256, 256, 256, 256, 256, 512, 512, 512, 512, 512]
kernel_size = [6, 9, 12, 6, 6, 9, 6, 12, 9, 12, 12, 6, 9, 6, 3, 6, 12, 9, 3, 3, 9, 9, 3, 6, 6, 6, 9, 6, 9, 9, 12, 3, 6, 6, 6, 6, 3]
stride_size = [4, 8, 8, 4, 4, 8, 8, 8, 1, 8, 8, 8, 1, 8, 4, 4, 2, 2, 4, 1, 8, 4, 2, 1, 4, 8, 1, 4, 1, 2, 8, 2, 4, 2, 2, 8, 2]
print(len(CNN_channel),len(dense_node), len(kernel_size),len(stride_size))

37 37 37 37


In [ ]:
import numpy as np
import pandas as pd
import esm
# whole dataset loading and dataset splitting 
dataset = pd.read_excel('allergens_dataset.xlsx',na_filter = False) # take care the NA sequence problem
# loading the y dataset for model development 
y = dataset['label']
y = np.array(y) # transformed as np.array for CNN model

# read the peptide embeddings
X_data_name = 'whole_sample_dataset_esm2_t36_3B_UR50D_unified_2560_dimension.csv'
X_data = pd.read_csv(X_data_name,header=0, index_col = 0,delimiter=',')
X = np.array(X_data)

ACC_repeat_col=[]
BACC_repeat_col = []
Sn_repeat_col= []
Sp_repeat_col=[]
MCC_repeat_col=[]
AUC_repeat_col=[]

for i in range(len(CNN_channel)):
  # split dataset as training and test dataset as ratio of 8:2
  ACC_collecton = []
  BACC_collecton = []
  Sn_collecton = []
  Sp_collecton = []
  MCC_collecton = []
  AUC_collecton = []
  for random_num in range(10):
    from sklearn.model_selection import train_test_split
    X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.2, random_state=random_num)

    # normalize the X data range
    from sklearn.preprocessing import MinMaxScaler
    scaler = MinMaxScaler()
    scaler.fit(X_train)
    X_train = scaler.transform(X_train) # normalize X to 0-1 range 
    X_test = scaler.transform(X_test)

    inputShape=(2560,1) # input feature size 
    input = Input(inputShape)
    x = Conv1D(CNN_channel[i],(kernel_size[i]),strides = (stride_size[i]),name='layer_conv2',padding='same')(input)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = MaxPooling1D((2), name='MaxPool2',padding="same")(x)
    x = Dropout(0.15)(x)
    x = Flatten()(x)
    x = Dense(dense_node[i],activation = 'relu',name='fc1')(x)
    x = Dropout(0.15)(x)
    x = Dense(2,activation = 'softmax',name='fc2')(x)
    model = Model(inputs = input,outputs = x,name='Predict')
    # define SGD optimizer
    momentum = 0.5
    sgd = SGD(lr=0.01, momentum=momentum, decay=0.0, nesterov=False)
    # compile the model
    model.compile(loss='sparse_categorical_crossentropy',optimizer=sgd, metrics=['accuracy'])
    # learning deccay setting
    import math
    def step_decay(epoch): # gradually decrease the learning rate 
        initial_lrate=0.1
        drop=0.6
        epochs_drop = 3.0
        lrate= initial_lrate * math.pow(drop,    # math.pow base raised to a power
              math.floor((1+epoch)/epochs_drop)) # math.floor Round numbers down to the nearest integer
        return lrate
    lr = LearningRateScheduler(step_decay)
    # early stop setting
    early_stop = EarlyStopping(monitor='val_accuracy', patience = 40,verbose=0,restore_best_weights = True)
    # set checkpoint and save the best model
    mc = ModelCheckpoint('best_modelbest_2560_repeat.h5',  monitor='val_accuracy', mode='max', verbose=0, save_best_only=True, save_weights_only=False)
    # summary the callbacks_list
    callbacks_list = [ lr , early_stop, mc]
    model_history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=200,callbacks=callbacks_list,batch_size = 16, verbose=0)
    # load the save best model
    saved_model = load_model('best_modelbest_2560_repeat.h5')
    # result collection list
    # confusion matrix 
    predicted_class= []
    predicted_protability = saved_model.predict(X_test,batch_size=1)
    for p in range(predicted_protability.shape[0]):
      index = np.where(predicted_protability[p] == np.amax(predicted_protability[p]))[0][0]
      predicted_class.append(index)
    predicted_class = np.array(predicted_class)
    y_true = y_test    
    from sklearn.metrics import confusion_matrix
    import math
    # np.ravel() return a flatten 1D array
    TP, FP, FN, TN = confusion_matrix(y_true, predicted_class).ravel() # shape [ [True-Positive, False-positive], [False-negative, True-negative] ]
    ACC = (TP+TN)/(TP+TN+FP+FN)
    if ACC>0.6:
      ACC_collecton.append(ACC)
      Sn_collecton.append(TP/(TP+FN))
      Sp_collecton.append(TN/(TN+FP))
      MCC = (TP*TN-FP*FN)/math.pow(((TP+FP)*(TP+FN)*(TN+FP)*(TN+FN)),0.5)
      MCC_collecton.append(MCC)
      BACC_collecton.append(0.5*TP/(TP+FN)+0.5*TN/(TN+FP))
      from sklearn.metrics import roc_auc_score
      AUC = roc_auc_score(y_test, predicted_protability[:,1])
      AUC_collecton.append(AUC)
  import statistics
  ACC_repeat_col.append(str(round(statistics.mean(ACC_collecton),3))+'±'+ str(round(statistics.stdev(ACC_collecton),3)))
  BACC_repeat_col.append(str(round(statistics.mean(BACC_collecton),3))+'±'+str(round(statistics.stdev(BACC_collecton),3)))
  Sn_repeat_col.append(str(round(statistics.mean(Sn_collecton),3))+'±'+str(round(statistics.stdev(Sn_collecton),3)))
  Sp_repeat_col.append(str(round(statistics.mean(Sp_collecton),3))+'±'+str(round(statistics.stdev(Sp_collecton),3)))
  MCC_repeat_col.append(str(round(statistics.mean(MCC_collecton),3))+'±'+str(round(statistics.stdev(MCC_collecton),3)))
  AUC_repeat_col.append(str(round(statistics.mean(AUC_collecton),3))+'±'+str(round(statistics.stdev(AUC_collecton),3)))
  print(ACC_repeat_col[i],BACC_repeat_col[i],Sn_repeat_col[i],Sp_repeat_col[i],MCC_repeat_col[i],AUC_repeat_col[i])

In [ ]:
# combine the lists into a list of tuples
combined_list = list(zip(ACC_repeat_col, BACC_repeat_col, Sn_repeat_col, Sp_repeat_col, MCC_repeat_col, AUC_repeat_col))

# create a DataFrame from the list of tuples
df = pd.DataFrame(combined_list, columns=['ACC_collection', 'BACC_collection', 'Sn_collection', 'Sp_collection', 'MCC_collection', 'AUC_collection'])

# export the DataFrame to an Excel file
df.to_excel('2560 ten times repeat performance output.xlsx', index=False)